In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [5]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
import pickle

data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('artifacts\label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('artifacts\onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('artifacts\scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [8]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model


In [9]:
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [10]:

# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [11]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


f:\pythoncode\ML\ML Projects\Churn-Modelling-using-ANN\venv\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
f:\pythoncode\ML\ML Projects\Churn-Modelling-using-ANN\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6531 - loss: 0.6209  
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 847us/step - accuracy: 0.8046 - loss: 0.4418
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - accuracy: 0.8148 - loss: 0.4208
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 804us/step - accuracy: 0.8310 - loss: 0.3998
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step - accuracy: 0.8345 - loss: 0.3988
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 781us/step - accuracy: 0.8323 - loss: 0.3847
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step - accuracy: 0.8396 - loss: 0.3765
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 804us/step - accuracy: 0.8448 - loss: 0.3738
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 799us/step - accuracy: 0.8471 - loss: 0.3632
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 786us/step - accuracy: 0.8615 - loss: 0.3430
Epoch 11/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 941us/step - accuracy: 0.8508 - loss: 0.3557
Epoch 12/50
250/250 ━━━━━━━━━━